In [1]:
# used to widen the cells 
from IPython.core.display import display, HTML
display(HTML("<style>.container { margin-left: 2.5% !important; width:95%; }</style>"))


In [5]:
from algosdk.v2client import algod
from algosdk import mnemonic
from algosdk import transaction

algod_address = "https://testnet-algorand.api.purestake.io/ps2"
algod_token = "fFG6R4JTFq9Hh8RjZy130aMRdUhmOYp68hi9nm4L"
headers = {
   "X-API-Key": algod_token,
}

algod_client = algod.AlgodClient(algod_token, algod_address, headers);

In [7]:
from algosdk import account 

In [8]:
# generate an account
private_key, address = account.generate_account()
print("Private key:", private_key)
print("Address:", address)
print("memo:", mnemonic.from_private_key(private_key))

Private key: AezFSrIDpntBtOZ5BkjBlFZOpn4EWV+9iyd2A7WLBNxp4lya6D2VXf6etYWCUEQf7WUZqB5D0LUYbOKjR5Y87Q==
Address: NHRFZGXIHWKV37U6WWCYEUCED7WWKGNIDZB5BNIYNTRKGR4WHTWW7XLMZI
memo: lens bless century depth equal galaxy pear trade song cactus gaze harsh original praise catalog flip program confirm because danger head carry then absorb sock


In [9]:
memo = "lens bless century depth equal galaxy pear trade song cactus gaze harsh original praise catalog flip program confirm because danger head carry then absorb sock"
private_key = mnemonic.to_private_key(memo)
address = mnemonic.to_public_key(memo)
print(address)

NHRFZGXIHWKV37U6WWCYEUCED7WWKGNIDZB5BNIYNTRKGR4WHTWW7XLMZI


In [2]:
def compile(approval_filename='approval.teal', clear_state_filename='clear_state.teal'):

    # pyteal to teal 
    with open(approval_filename, 'w') as f:
        compiled = compileTeal(approval_program(), Mode.Application)
        f.write(compiled)

    with open(clear_state_filename, 'w') as f:
        compiled = compileTeal(clear_state_program(), Mode.Application)
        f.write(compiled)

    # teal to bytecode 
    stdout, stderr = execute(["goal", "clerk", "compile", "-o", approval_filename+'.tok', approval_filename])
    if stderr != "":
        print(stderr)
        raise
    elif len(stdout) < 59:
        print("error in compile teal")
        raise

    stdout, stderr = execute(["goal", "clerk", "compile", "-o", clear_state_filename+'.tok', clear_state_filename])
    if stderr != "":
        print(stderr)
        raise
    elif len(stdout) < 59:
        print("error in compile teal")
        raise

    with open(approval_filename+'.tok', 'rb') as f:
        approval_bytes = f.read()

    with open(clear_state_filename+'.tok', 'rb') as f: 
        clear_state_bytes = f.read() 
        
    return approval_bytes, clear_state_bytes

In [16]:
# read app global state
def read_global_state(client, addr, app_id):   
    results = client.account_info(addr)
    apps_created = results['created-apps']
    global_states = []
    for app in apps_created :
        if app['id'] == app_id :
            global_states.append(app['params']['global-state'])
            # print(f"global_state for app_id {app_id}: ", app['params']['global-state'])
    return global_states 

In [44]:
import base64 
def decode_keys(s):
    for entry in s[0]:
        entry['key'] = base64.b64decode(entry['key']).decode('utf-8')
    return s

In [81]:
from pyteal import *    
    
def approval_program():     
    separator = Bytes("::")
    
    on_creation = Seq([    
        App.globalPut(Bytes("STORE"), Bytes("")),  
        Return(Int(1))
    ])    
    
    on_optin = Return(Int(1))
    
    store = App.globalGet(Bytes("STORE"))
    time_stamp = Itob(Global.latest_timestamp())
    on_push = Seq([
        # App.globalPut(Bytes("timestamp"), Global.latest_timestamp()),
        # App.globalPut(Bytes("roudn"), Global.round()),
        
        App.globalPut(Bytes('STORE'), Concat(store, separator, time_stamp)),    # store the timestamp, which is also the key to the app argument in the correct order 
        App.globalPut(time_stamp, Txn.application_args[1]),     # store the contract address, indexed by the timestamp 
        Return(Int(1))
    ])
      
    program = Cond(    
            [Txn.application_id() == Int(0), on_creation],    
            [Txn.on_completion() == OnComplete.DeleteApplication, Return(Int(0))], 
            [Txn.on_completion() == OnComplete.UpdateApplication, Return(Int(0))],    
            [Txn.on_completion() == OnComplete.OptIn, on_optin],
            [Txn.application_args[0] == Bytes("push"), on_push],
    )    
    return program    
    
    
def clear_state_program():     
    program = Return(Int(1))
    return program    


In [82]:
from algosdk.future import transaction
from algosdk.future.transaction import StateSchema
from algosdk.future.transaction import OnComplete as onComplete

In [83]:
params = algod_client.suggested_params()
params.falt_fee = True
params.fee = 1000

In [91]:
approval_bytes, clear_state_bytes = compile()

local_schema = StateSchema(num_uints=1, num_byte_slices=1)    
global_schema = StateSchema(num_uints=5, num_byte_slices=5)   

on_complete = onComplete(0)
app_create_txn = transaction.ApplicationCreateTxn(
    address, 
    params, 
    on_complete, 
    approval_bytes, 
    clear_state_bytes, 
    global_schema, 
    local_schema,
)

app_create_txn_signed = app_create_txn.sign(private_key)
txid = algod_client.send_transaction(app_create_txn_signed)
txid

'WPU3VIC6VSQSFAUXHSLB24AC6SYAK5ELG7PLXJJ3DN5SEUOK4XEQ'

In [97]:
app_id = algod_client.pending_transaction_info(txid)['application-index']
app_id

14877600

In [102]:
global_state = read_global_state(algod_client, address, app_id)
state = decode_keys(global_state)
print(state)
print(len(state[0][0]['value']['bytes']))

[[{'key': 'STORE', 'value': {'bytes': 'MTIzNDU2Nzg5', 'type': 1, 'uint': 0}}, {'key': 'timestamp', 'value': {'bytes': '', 'type': 2, 'uint': 1616270061}}, {'key': 'roudn', 'value': {'bytes': '', 'type': 2, 'uint': 13035730}}]]
12


In [103]:
len('1616270061')

10

In [104]:
len('13035730')

8

In [105]:
#### NOTE we could store the first few digits separately, since they change much more slowly
# example: blocknumbers are 125442 125444 125450, we can store 1254 42 44 50 which is shorter. We know that if we encounter a number of 4 digits, it's a prefix
# they will be in sorted order, so when we push we just check the next one, if the length is the length of the prefix, we push that too. 
# when adding we simply check to see if the prefix portion of the blocknumber is different 

In [99]:
optin_tx = transaction.ApplicationOptInTxn(address, params, app_id)
algod_client.send_transaction(optin_tx.sign(private_key))

'BREW5PVGZOPFH2LCWRIIXHT5RJS3SPHS2NQOGF6IKZIJD6A67LEA'

In [100]:
s = "123456789"
app_args = ["push".encode("utf-8"), s.encode("utf-8")]

# create unsigned transaction
call_tx = transaction.ApplicationNoOpTxn(address, params, app_id, app_args)
call_txid = algod_client.send_transaction(call_tx.sign(private_key))

In [79]:
params = algod_client.suggested_params()
params.falt_fee = True
params.fee = 1000